In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import numpy as np
from pandas.core.frame import DataFrame

In [2]:
path = ('/Users/njxwq/Desktop/ipynb/big_data/measurement_data') #define the path where all data is stored

In [3]:
def get_folders(path): #read all the folders in the path
    folders =os.listdir(path)
    folders.sort()
    list= []
    for folder in folders:
        if not  os.path.isdir(path +folder):        
            f_name = str(folder)
            tr = '/'   
            filename = path + tr + f_name
            if len(f_name) == 10:
                list.append(filename) 
    return list

In [4]:
get_folders(path) #check all the folders

['/Users/njxwq/Desktop/ipynb/big_data/measurement_data/Bearing1_4',
 '/Users/njxwq/Desktop/ipynb/big_data/measurement_data/Bearing1_5',
 '/Users/njxwq/Desktop/ipynb/big_data/measurement_data/Bearing1_6',
 '/Users/njxwq/Desktop/ipynb/big_data/measurement_data/Bearing1_7',
 '/Users/njxwq/Desktop/ipynb/big_data/measurement_data/Bearing2_4',
 '/Users/njxwq/Desktop/ipynb/big_data/measurement_data/Bearing2_5',
 '/Users/njxwq/Desktop/ipynb/big_data/measurement_data/Bearing2_6',
 '/Users/njxwq/Desktop/ipynb/big_data/measurement_data/Bearing2_7',
 '/Users/njxwq/Desktop/ipynb/big_data/measurement_data/Bearing3_3']

In [5]:
def get_files(path): #read all the files in one folder, store the features data and label data separately
    files =os.listdir(path)
    files.sort()
    list= []
    list1 = []
    for file in files:
        if not  os.path.isdir(path +file):        
            f_name = str(file)
            tr = '/'   
            filename = path + tr + f_name
            name, typename = filename.split('.')
            if typename == 'csv':
                if len(f_name) == 13:
                    list.append(filename)
                else:
                    list1.append(filename)
    return list,list1

In [6]:
def mean_median(list):  #collect the mean and mid value of features hori_acc and ver_acc
    list_datatime = []
    list_mean_hori = []
    list_mean_ver = []
    list_mid_hori = []
    list_mid_ver = []
    for file in list:
        df1 = pd.read_csv(file)
        df = transform_new(df1)
        list_mean_hori.append(np.mean(df['hori_acc']))
        list_mean_ver.append(np.mean(df['ver_acc']))
        list_mid_hori.append(np.median(df['hori_acc']))
        list_mid_ver.append(np.median(df['ver_acc']))
        list_datatime.append(df['datatime'][0])
    
    return list_mean_hori, list_mean_ver, list_mid_hori, list_mid_ver, list_datatime

In [7]:
def transform_new(data): #transform the input data to suitable form. 
    if len(data.columns) == 1:
        data = data.iloc[:,0].str.split(';', expand = True)
        data.columns = ['hour', 'minute', 'second', 'microsecond', 'hori_acc', 'ver_acc']
        data[['hour', 'minute','second',  'microsecond', 'hori_acc', 'ver_acc']] = data[['hour', 'minute','second', 'microsecond', 'hori_acc', 'ver_acc']].astype(float)
        data['datatime'] = data['hour'] + data['minute']/60 + data['second']/3600
        data.drop('hour', axis = 1, inplace = True)
        data.drop('minute', axis = 1, inplace = True)
        data.drop('second', axis = 1, inplace = True)
        data.drop('microsecond', axis = 1, inplace = True)

        data.insert(0, 'datatime', data.pop('datatime'))
    else:
        data.columns = ['hour', 'minute', 'second', 'microsecond', 'hori_acc', 'ver_acc']
        data[['hour', 'minute','second',  'microsecond', 'hori_acc', 'ver_acc']] = data[['hour', 'minute','second', 'microsecond', 'hori_acc', 'ver_acc']].astype(float)
        data['datatime'] = data['hour'] + data['minute']/60 + data['second']/3600
        data.drop('hour', axis = 1, inplace = True)
        data.drop('minute', axis = 1, inplace = True)
        data.drop('second', axis = 1, inplace = True)
        data.drop('microsecond', axis = 1, inplace = True)
    return data

In [8]:
def mean_median(list):  #collect the mean and mid value of features (hori_acc and ver_acc)
    list_datatime = []
    list_mean_hori = []
    list_mean_ver = []
    list_mid_hori = []
    list_mid_ver = []
    for file in list:
        df3 = pd.read_csv(file)
        df4 = transform_new(df3)
        list_mean_hori.append(np.mean(df4['hori_acc']))
        list_mean_ver.append(np.mean(df4['ver_acc']))
        list_mid_hori.append(np.median(df4['hori_acc']))
        list_mid_ver.append(np.median(df4['ver_acc']))
        list_datatime.append(df4['datatime'][0])
    
    return list_mean_hori, list_mean_ver, list_mid_hori, list_mid_ver, list_datatime


In [9]:
list_folder = get_folders(path)  
list_df = []  #create a list which stores the dataframe from each file

In [10]:
for path1 in list_folder:   #from all the data files transform dataframe and store it in the pre-defined list
    list_file, list_label = get_files(path1)
    list_mean_hori, list_mean_ver, list_mid_hori, list_mid_ver, list_datatime = mean_median(list_file)
    dict1 = {'mean_hori': list_mean_hori, 'mean_ver': list_mean_ver, 'median_hori': list_mid_hori, 'median_ver': list_mid_ver, 'datatime': list_datatime}
    df = DataFrame(dict1)
    df2 = pd.read_csv(list_label[0])
    df['state'] = df2['state']
    df['file'] = df2['file']
    df.insert(0, 'file', df.pop('file'))
    df.insert(1, 'datatime', df.pop('datatime'))
    list_df.append(df)

In [11]:
df = list_df[0]  #combine all the dataframes and use only one dataframe to represent all the data.
for i in range(1, len(list_df)):
    df = pd.concat([df, list_df[i]], ignore_index = True)

In [12]:
df #check the new dataframe which will be used in classfication

,file,datatime,mean_hori,mean_ver,median_hori,median_ver,state
0,acc_00001.csv,8.133333,0.006363,0.001671,0.003,0.014,0
1,acc_00002.csv,8.136111,-0.009000,0.006691,-0.013,0.012,0
2,acc_00003.csv,8.138889,-0.006220,-0.008295,-0.011,0.005,0
3,acc_00004.csv,8.141667,-0.005819,-0.001751,0.002,0.009,0
4,acc_00005.csv,8.144444,-0.002016,0.006631,-0.009,0.002,0
...,...,...,...,...,...,...,...
13020,acc_00430.csv,9.650833,-0.039199,0.003162,0.008,-0.008,2
13021,acc_00431.csv,9.653611,-0.029962,-0.002062,0.022,-0.023,2
13022,acc_00432.csv,9.656389,-0.056482,0.050883,-0.036,-0.032,2
13023,acc_00433.csv,9.659167,0.013096,0.028659,0.059,-0.050,2


In [13]:
df.drop('file', axis = 1, inplace = True)   #drop some unimportant colunmns which are useless in training process
df.drop('datatime', axis = 1, inplace = True)
df

,mean_hori,mean_ver,median_hori,median_ver,state
0,0.006363,0.001671,0.003,0.014,0
1,-0.009000,0.006691,-0.013,0.012,0
2,-0.006220,-0.008295,-0.011,0.005,0
3,-0.005819,-0.001751,0.002,0.009,0
4,-0.002016,0.006631,-0.009,0.002,0
...,...,...,...,...,...
13020,-0.039199,0.003162,0.008,-0.008,2
13021,-0.029962,-0.002062,0.022,-0.023,2
13022,-0.056482,0.050883,-0.036,-0.032,2
13023,0.013096,0.028659,0.059,-0.050,2


In [14]:
df['state'].value_counts()  #check the component of label data. The data is a little imbalanced.

1    9116
0    1959
2    1950
Name: state, dtype: int64

In [15]:
df.to_csv('the_whole_data.csv') #store the transformed dataframe in order to be used in the following task.